In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should display your GPU's name


True
NVIDIA GeForce RTX 3080


In [2]:
%pip install llama-index-extractors-entity

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install span-marker

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
from span_marker import SpanMarkerModel

model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")



c:\Users\Muthu\.conda\envs\llamaindex\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import SentenceSplitter

entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,  # include the entity label in the metadata (can be erroneous)
    device="cuda",  # set to "cuda" if you have a GPU or "cpu" if you have CPU alone
    model_name="tomaarsen/span-marker-mbert-base-multinerd",  # Local model name
    entity_types=["PERSON", "ORG", "GPE"],
)




c:\Users\Muthu\.conda\envs\llamaindex\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in EntityExtractor has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
from llama_index.core import Document

test_document_1 = Document(text="""
                        LLMs offer a natural language interface between humans and data. LLMs come pre-trained on huge amounts of publicly available data, but they are not trained on your data. Your data may be private or specific to the problem you're trying to solve. It's behind APIs, in SQL databases, or trapped in PDFs and slide decks.
                        Context augmentation makes your data available to the LLM to solve the problem at hand. LlamaIndex provides the tools to build any of context-augmentation use case, from prototype to production. Our tools allow you to ingest, parse, index and process your data and quickly implement complex query workflows combining data access with LLM prompting.
                        The most popular example of context-augmentation is Retrieval-Augmented Generation or RAG, which combines context with LLMs at inference time.""")
test_document_2 = Document(text=""" 
                        Agents are LLM-powered knowledge assistants that use tools to perform tasks like research, data extraction, and more. Agents range from simple question-answering to being able to sense, decide and take actions in order to complete tasks.
                        LlamaIndex provides a framework for building agents including the ability to use RAG pipelines as one of many tools to complete a task.""")

test_documents = [test_document_1,test_document_2]




In [ ]:
import nltk

# Redownload the punkt resource
nltk.download('punkt_tab')

In [9]:

entity_result = await entity_extractor.aprocess_nodes(test_documents)



Extracting entities: 100%|██████████| 2/2 [00:00<00:00,  3.11it/s]


In [10]:
entity_result

[Document(id_='f1460642-b13a-43bd-8791-29e0fb379f5d', embedding=None, metadata={'entities': ['LlamaIndex', 'SQL']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="\n                        LLMs offer a natural language interface between humans and data. LLMs come pre-trained on huge amounts of publicly available data, but they are not trained on your data. Your data may be private or specific to the problem you're trying to solve. It's behind APIs, in SQL databases, or trapped in PDFs and slide decks.\n                        Context augmentation makes your data available to the LLM to solve the problem at hand. LlamaIndex provides the tools to build any of context-augmentation use case, from prototype to production. Our tools allow you to ingest, parse, index and process your data and quickly implement complex query workflows combining data access with LLM prompting.\n                

In [11]:

print("\nEntity Output:")
for node in entity_result:
    print(f"Node ID: {node.node_id}, Entities: {node.metadata.get('entities', 'No entities available')}")


Entity Output:
Node ID: f1460642-b13a-43bd-8791-29e0fb379f5d, Entities: ['LlamaIndex', 'SQL']
Node ID: 60631b38-f9ff-4587-b066-6c38a8ea4e72, Entities: ['LlamaIndex']


In [12]:
node_parser = SentenceSplitter()

transformations = [node_parser, entity_extractor]

In [13]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

nodes = pipeline.run(documents=test_documents)

Extracting entities: 100%|██████████| 2/2 [00:00<00:00,  5.06it/s]


In [14]:
nodes

[TextNode(id_='b150cb62-900e-4cb8-9a5c-0a6a82e5f3e7', embedding=None, metadata={'entities': ['SQL']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f1460642-b13a-43bd-8791-29e0fb379f5d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'entities': ['LlamaIndex', 'SQL']}, hash='cc6b43aa8335c6efe1e351005bb7ee363a12e227222b42d4682c185062d7ae45')}, metadata_template='{key}: {value}', metadata_separator='\n', text="LLMs offer a natural language interface between humans and data. LLMs come pre-trained on huge amounts of publicly available data, but they are not trained on your data. Your data may be private or specific to the problem you're trying to solve. It's behind APIs, in SQL databases, or trapped in PDFs and slide decks.\n                        Context augmentation makes your data available to the LLM to solve the problem at hand. LlamaIndex provides the tools to build any of context-augmentation

In [15]:
for node in nodes:
    print(f"Node ID: {node.node_id}, Entities: {node.metadata.get('entities', 'No entities available')}")
    print("======================================================")


Node ID: b150cb62-900e-4cb8-9a5c-0a6a82e5f3e7, Entities: ['SQL']
Node ID: a109fc78-9365-4447-b9cf-170fbb358e27, Entities: ['LlamaIndex']


In [16]:
from llama_index.core import SimpleDirectoryReader

#documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()
documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()


In [17]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=50),  # Split text into manageable chunks
        entity_extractor,  # Extract summaries
    ]
)


In [18]:
nodes = pipeline.run(
    documents=documents,
    in_place=True,
    show_progress=True
)


Extracting entities: 100%|██████████| 18/18 [00:03<00:00,  5.58it/s]


In [ ]:
nodes

In [19]:
for node in nodes:
    print(f"Node ID: {node.node_id}, Entities: {node.metadata.get('entities', 'No entities available')}")
    print("======================================================")


Node ID: 17cc4217-2460-4b4c-87ad-8a814fe5b97d, Entities: ['Rich Draves', 'IBM']
Node ID: 98310f07-0c47-44cf-a60b-7dd60515223a, Entities: ['Winograd', 'Cornell']
Node ID: 06476864-c896-44fd-9544-d9013c93d3e7, Entities: ['Rich Draves', 'Xerox Dandelions', 'Carnegie Institute', 'CMU']
Node ID: cc5cc40e-acae-45fd-b8a2-905e1559037d, Entities: ['Cezanne', 'Florence']
Node ID: 96e41351-5d7c-4df7-81bf-7ca509adb524, Entities: ['Ulivi', 'Interleaf', 'Microsoft Word', 'Italian']
Node ID: e3d5a610-e770-4017-b08d-9579f92cba02, Entities: ['Roy Lichtenstein']
Node ID: 7ca997dd-830f-4b41-bb0a-7d19274a520b, Entities: ['Y Combinator', 'Robert']
Node ID: 4b53d6f2-2299-4450-aaf5-820fa9e99da8, Entities: ['Robert', 'Trevor', 'Trevor Blackwell']
Node ID: 739470d7-5c35-4b5d-931f-9fe177909f32, Entities: ['Y Combinator']
Node ID: a20ba74b-663e-4e04-a654-41e2e1767755, Entities: ['California', 'New York', 'Santa Cruz Mountains']
Node ID: 63816175-e7a3-43a4-8f44-f9e3157302d1, Entities: ['Y Combinator', 'Lisp']
Nod